# Simple oil spill diffusion problem

<img src="/img_oilspill.png" width=300 height=200/>

## Information
This problem is about a simulated oil spill occurring in a still ocean. It uses the same framework as the code oilspill_diffusion_ftcs.ipynb, but here we are using a fully-implicit scheme. First, let's recap the problem.

The oil will progressively diffuse away from the spill location. We can solve the diffusion equation numerically to compute the oil concentration at the ocean surface as a function of space and time.
In this problem, we simulate only diffusion, and we neglect advection by ocean currents. This is not realistic, but could be a better approximation for other cases (e.g., spill in an aquifer).

The diffusion equation is: 

$\frac{\partial u(x)}{\partial t}=D\frac{\partial^{2}u(x)}{\partial x^{2}}$ 

Where,
- $u(x)$: the oil concentration kg [$kg$ $m^{-3}$]
- $D$: the oil diffusion coefficient [$m^{2}$ $s^{-1}$] 

We fix the parameter $D$ to 10 $m^{2}$ $s^{-1}$, following Zelenke et al. (2012). We use a 100 km long simulation domain, and simulate the diffusion of oil from its starting point $x=0$.

We use a an implicit scheme, with centered differences in space. Under this framework, the diffusion equation is discretized as (see derivation in class):

$\frac{u_{j}^{(n+1)}-u_{j}^{(n)}}{\Delta t}=D\frac{u_{j+1}^{(n+1)}-2u_{j}^{(n+1)}+u_{j-1}^{(n+1)}}{\Delta x^{2}}$,

which leads to:

$-\alpha u_{j-1}^{(n+1)} + (1+2\alpha) u_{j}^{(n+1)} - \alpha u_{j+1}^{(n+1)} = u_{j}^{(n)}$,

where

$\alpha=\frac{D\Delta t}{\Delta x^{2}}$

As in oilspill_diffusion_ftcs.ipynb, you should think about the implementation of boundary conditions, the equilibrium profile, and how ocean currents can be included. Using the code specific to this notebook, you should think about the following aspects:
- How does the solution method differs from the FTCS scheme?
- Can you test the numerical stability of this implicit scheme?
- What is the difference between numerical stability and accuracy?

In [ ]:
import os
import sys
import math
import copy
import time
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import diags

Set up the x-grid.

In [ ]:
deltax      = 1000.0 #[m]
xgrid       = np.arange(0,1e5,deltax) #[m]
xnum        = len(xgrid)
spill_scale = 1000.0 #[m]

Define the initial values. Here, we simulate a release of oil similar in magnitude to the Deepwater Horizon oil spill of 2010$^{\text{(a)}}$. We assume that the oil is released over a surface of 1 $km^{2}$, and down to a depth f 1 $m$. Finally, we assume a very low baseline of oil concentration in the ocean.

In [ ]:
uu_base   = 0.001 #[kg m-3] baseline oil pollution concentration at the ocean surface
uu_spill  = 560000/(1*spill_scale**2) #[kg m-3] oil concentration at the spill location (Deepwater Horizon value, wiki)

Define the boundary conditions.

In [ ]:
uu_left_dirich  = copy.deepcopy(uu_spill)
uu_right_dirich = copy.deepcopy(uu_base)

Define the parameters.

In [ ]:
### Diffusion coefficient ###
bigd      = 10.0 #[m2 s-1] (section 8 of Zelenke et al., 2012)

### Time parameters ###
secperd    = 3600*24 #[s]
tmend_d    = 365 #days
deltat_d   = 0.1 #[days]
nsteps     = int(tmend_d/deltat_d)
deltat_sec = deltat_d*secperd
ttplot     = np.linspace(0.001*nsteps,nsteps-1,7).astype(int).tolist()

Initialize our arrays for the solutions.

In [ ]:
uu_0   = uu_base*np.ones(xnum)
uu_0   = uu_0 + uu_spill*np.exp(-(1/2)*xgrid**2/spill_scale**2) #Gaussian profile
uu_cur = copy.deepcopy(uu_0)

Define the $H$ matrix.

In [ ]:
alpha         = bigd*deltat_sec/deltax**2
updiag        = -alpha*np.ones(xnum-1)
updiag[0]     = 0
maindiag      = (1+2*alpha)*np.ones(xnum)
maindiag[0]   = 1
maindiag[-1]  = 1
lowdiag       = -alpha*np.ones(xnum-1)
lowdiag[-1]   = 0
hmat          = diags([lowdiag,maindiag,updiag],[-1,0,1]).toarray()
invhmat       = np.linalg.inv(hmat)

Loop in time

In [ ]:
to_plot = np.zeros((len(ttplot),xnum))
for tt in range(nsteps):
    # Copy old uu field #
    uu_old = copy.deepcopy(uu_cur)
    # Solve matrix system #
    uu_old = uu_old.reshape((xnum,1))
    uu_cur = (invhmat @ uu_old)
    uu_cur = uu_cur.flatten()
    # Save for plotting #
    if(tt in ttplot):
        to_plot[ttplot.index(tt),:] = copy.deepcopy(uu_cur)

And make a figure.

In [ ]:
plcols = plt.cm.jet
fig = plt.figure(figsize=(10,6))
ax = plt.subplot(111)
for kk in range(len(ttplot)):
    ax.plot(xgrid,to_plot[kk,:],c=plcols(kk/len(ttplot)),label=f'{np.round((ttplot[kk]+1)*deltat_d,1)} days')
ax.legend(loc='best',fontsize=12)
ax.tick_params(which='major',axis='both',labelsize=12)    
ax.set_xlabel('x [m]',fontsize=12)
ax.set_ylabel(r'Concentration $u(x,t)$ [$kg$ $m^{-3}$]',fontsize=12)
fig.tight_layout()